## 로컬 환경에서 PDF 파일 RAG 검색하기 3단계

__step3__
- PDF 문서 여러개 로드하여 임베딩 후 csv 파일로 저장
- csv 파일을 랭체인 FAISS 인덱싱하여 인덱스 파일 생성 및 저장
- 랭체인 프레임워크 적용하여 llm 검색 까지 구현

In [3]:
import os
import csv
import json
import ast
import faiss
import numpy as np
import pandas as pd
from langchain.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings
from sentence_transformers import SentenceTransformer

# 1. 문서 로드
loader = DirectoryLoader(
    'data',                         
    glob='*.pdf',                   
    loader_cls=PyMuPDFLoader        
)
docs = loader.load()

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 3. 임베딩을 위한 클래스 생성
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)
    
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()
    
    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]

embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")

# 4. 임베딩 저장 함수

def save_embeddings_to_csv(documents, embedding_model, folder_path):
    os.makedirs(folder_path, exist_ok=True)
    
    file_docs = {}
    for doc in documents:
        file_name = os.path.basename(doc.metadata['source']).replace('.pdf', '')
        if file_name not in file_docs:
            file_docs[file_name] = []
        file_docs[file_name].append(doc)
    
    for file_name, docs in file_docs.items():
        full_path = os.path.join(folder_path, f"{file_name}.csv")
        
        embeddings = embedding_model.embed_documents([doc.page_content for doc in docs])
        
        with open(full_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["document", "embedding"])
            
            for doc, embedding in zip(docs, embeddings):
                writer.writerow([doc.page_content, json.dumps(embedding)])  # JSON 형식으로 저장
        
        print(f"✅ 임베딩 데이터가 {full_path} 파일에 저장되었습니다.")
    
save_embeddings_to_csv(split_documents, embedding_model, 'csv/')

# 5. CSV에서 임베딩 로드 및 FAISS 인덱스 생성

def load_csv_embeddings(folder_path):
    data_dict = {}
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            
            if "document" in df.columns and "embedding" in df.columns:
                documents, embeddings, metadatas = [], [], []
                for _, row in df.iterrows():
                    text = row["document"]
                    try:
                        embedding = json.loads(row["embedding"])  # JSON 로드 방식으로 변경
                        if isinstance(embedding, list):
                            embeddings.append(np.array(embedding, dtype=np.float32))
                            documents.append(text)
                            metadatas.append({"source": filename})
                        else:
                            print(f"⚠️ {filename}의 임베딩 형식이 올바르지 않습니다!")
                    except Exception as e:
                        print(f"⚠️ {filename}에서 임베딩 변환 오류: {e}")
                data_dict[filename.replace('.csv', '')] = (documents, embeddings, metadatas)
    
    return data_dict

# FAISS 인덱스 생성 및 저장

def create_faiss_indexes(data_dict, save_path):
    os.makedirs(save_path, exist_ok=True)
    
    for file_name, (documents, embeddings, metadatas) in data_dict.items():
        vector_store = FAISS.from_texts(texts=documents, embedding=embedding_model, metadatas=metadatas)
        faiss_index_path = os.path.join(save_path, file_name)
        vector_store.save_local(faiss_index_path)
        print(f"✅ {file_name}에 대한 FAISS 인덱스 저장 완료: {faiss_index_path}")

# 실행
if __name__ == "__main__":
    data_folder = "./csv"
    faiss_index_folder = "./faiss_index"
    
    data_dict = load_csv_embeddings(data_folder)
    print(f"📄 {len(data_dict)}개의 파일 로드 완료!")
    
    if data_dict:
        create_faiss_indexes(data_dict, faiss_index_folder)
    else:
        print("⚠️ 문서 또는 임베딩 데이터가 없습니다. FAISS 인덱스를 생성할 수 없습니다!")


✅ 임베딩 데이터가 csv/SPRI_AI_Brief_2023년12월호_F.csv 파일에 저장되었습니다.
✅ 임베딩 데이터가 csv/AI기반_인파분석플랫폼구축_제안서.csv 파일에 저장되었습니다.
✅ 임베딩 데이터가 csv/운영체제_중간과제물.csv 파일에 저장되었습니다.
📄 3개의 파일 로드 완료!
✅ SPRI_AI_Brief_2023년12월호_F에 대한 FAISS 인덱스 저장 완료: ./faiss_index/SPRI_AI_Brief_2023년12월호_F
✅ AI기반_인파분석플랫폼구축_제안서에 대한 FAISS 인덱스 저장 완료: ./faiss_index/AI기반_인파분석플랫폼구축_제안서
✅ 운영체제_중간과제물에 대한 FAISS 인덱스 저장 완료: ./faiss_index/운영체제_중간과제물


In [ ]:
def search_faiss_index(query, index_folder, file_name, top_k=5):
    index_path = os.path.join(index_folder, file_name)
    
    # FAISS 인덱스 로드 시 allow_dangerous_deserialization 옵션 추가
    vector_store = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)
    
    query_embedding = embedding_model.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=top_k)
    
    print(f"🔍 '{query}' 검색 결과:")
    for i, result in enumerate(results):
        print(f"{i+1}. {result.page_content} (출처: {result.metadata['source']})")
    
    return results

#search_faiss_index("포항에서 열리는 축제의 이름은 무엇인가?", "./faiss_index", "AI기반_인파분석플랫폼구축_제안서")
search_faiss_index("삼성에서 만든 AI의 이름은 무엇인가?", "./faiss_index", "SPRI_AI_Brief_2023년12월호_F")

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

import os

# ✅ Step 6: 프롬프트 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean, and make sure the answer ends with '입니다'.

#Context: 
{context}

#Question:
{question}

#Answer(Ensure the response ends with '입니다'):"""
)

# ✅ Step 7: 언어 모델 (LLM) 생성
llm = Ollama(model="llama3.2")  # 최신 모델명 확인 후 변경

# ✅ FAISS 인덱스 로드 (보안 옵션 추가)
faiss_index_folder = "./faiss_index"
faiss_indexes = {}

for file_name in os.listdir(faiss_index_folder):
    index_path = os.path.join(faiss_index_folder, file_name)
    if os.path.isdir(index_path):
        faiss_indexes[file_name] = FAISS.load_local(
            index_path,
            embedding_model,
            allow_dangerous_deserialization=True  # 🚨 보안 옵션 추가
        )

# ✅ 모든 인덱스를 검색하는 함수
def multi_index_search(question, faiss_indexes):
    all_docs = []
    
    # 각 FAISS 인덱스에서 검색
    for index_name, index in faiss_indexes.items():
        retriever = index.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # 🔹 FAISS 검색 최적화
        docs = retriever.get_relevant_documents(question)  # 각 인덱스에서 검색된 문서
        for doc in docs:
            doc.metadata["source"] = index_name  # 출처 정보 추가
        all_docs.extend(docs)

    if not all_docs:
        return "⚠️ 관련 문서를 찾을 수 없습니다."
    
    # 🔹 FAISS 검색 결과를 기반으로 RetrievalQA 생성
    retriever = FAISS.from_documents(all_docs, embedding_model).as_retriever()

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,  # ✅ 올바른 retriever 전달
        chain_type="stuff",
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )
    
    response = qa_chain.invoke({"query": question})  # 🔹 context 제거 후 수정
    
    answer = response.get("result", "⚠️ 답변을 생성할 수 없습니다.")  # key 수정
    sources = "\n".join(set([doc.metadata.get("source", "알 수 없음") for doc in all_docs]))

    return f"💬 답변: {answer}\n📌 출처: {sources}"

# 📌 사용 예시
question = "삼성전자에서 만든 ai 의 이름은?"
print(multi_index_search(question, faiss_indexes))


/var/folders/05/rphn83jd5hz6tgnm_w24b6zr0000gn/T/ipykernel_88128/643498905.py:47: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)  # 각 인덱스에서 검색된 문서


💬 답변: 삼성 가우스입니다.
📌 출처: faiss_index
운영체제_중간과제물
AI기반_인파분석플랫폼구축_제안서
SPRI_AI_Brief_2023년12월호_F
